# Gemini Sentiment Analysis Agent Demo


### Get API keys

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in [Google AI Studio](https://makersuite.google.com/app/apikey).  You'll also need an [OpenAI API key](https://openai.com/).

Once you have your API keys create the following Google Colab "secrets"
`GOOGLE_API_KEY` and `OPENAI_API_KEY` which will contain your respective keys.

## Initial Setup

In [ ]:
!pip install llama-index google-generativeai openai pypdf -q

In [2]:
import os
from google.colab import userdata
import openai

openai.api_key = userdata.get('OPENAI_API_KEY')

In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO, force=True)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Vectorize Mindfulness Guides

In [ ]:
import os.path
from llama_index import (
  VectorStoreIndex,
  SimpleDirectoryReader,
  StorageContext,
  load_index_from_storage,
  )

# Attempt to load embeddings
try:
  storage_context = StorageContext.from_defaults(
      persist_dir="./storage/challenging_child"
  )
  challenging_child_index = load_index_from_storage(storage_context)

  storage_context = StorageContext.from_defaults(
      persist_dir="./storage/mindfulness_TB_50"
  )
  mindfulness_TB_50_index = load_index_from_storage(storage_context)

  storage_context = StorageContext.from_defaults(
      persist_dir="./storage/mindfulness_TB_relationships"
  )
  mindfulness_TB_relationships_index = load_index_from_storage(storage_context)
except:
  # Need to generate embeddings
  challenging_child_docs = SimpleDirectoryReader(
      input_files=["./data/The_Challenging_Child_Toolbox.pdf"]
  ).load_data(show_progress=True)
  mindfulness_TB_50_docs = SimpleDirectoryReader(
      input_files=["./data/The_Mindfulness_Toolbox__50_Practical_Tips_Tools.pdf"]
  ).load_data(show_progress=True)
  mindfulness_TB_relationships_docs = SimpleDirectoryReader(
      input_files=["./data/The_Mindfulness_Toolbox_for_Relationships.pdf"]
  ).load_data(show_progress=True)

  # Build index
  challenging_child_index = VectorStoreIndex.from_documents(challenging_child_docs)
  mindfulness_TB_50_index = VectorStoreIndex.from_documents(mindfulness_TB_50_docs)
  mindfulness_TB_relationships_index = VectorStoreIndex.from_documents(mindfulness_TB_relationships_docs)

  # Persist
  challenging_child_index.storage_context.persist(persist_dir="./storage/challenging_child")
  mindfulness_TB_50_index.storage_context.persist("./storage/mindfulness_TB_50")
  mindfulness_TB_relationships_index.storage_context.persist(persist_dir="./storage/mindfulness_TB_relationships")

In [5]:
challenging_child_engine = challenging_child_index.as_query_engine()
mindfulness_TB_50_engine = mindfulness_TB_50_index.as_query_engine()
mindfulness_TB_relationships_engine = mindfulness_TB_relationships_index.as_query_engine()

### Test to make sure our data loaded

In [6]:
from IPython.display import Markdown

In [7]:
# The_Challenging_Child_Toolbox
query_engine = challenging_child_index.as_query_engine()
response = query_engine.query("What is the definition of a challenging child?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

A challenging child refers to a child who has emotional and behavioral difficulties. These difficulties may require special attention and support from caregivers or professionals working with the child.

In [8]:
# The_Mindfulness_Toolbox__50_Practical_Tips_Tools
query_engine = mindfulness_TB_50_index.as_query_engine()
response = query_engine.query("What are the benefits of a mindfulness practice?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Mindfulness practice has several benefits. It enhances flexibility and adaptability, allowing individuals to break free from old habits and retrain their brains. It cultivates curiosity and greater ease, encouraging exploration of the journey and process rather than being overly focused on the outcome. Mindfulness also changes one's relationship to self-critical and self-blaming thoughts, promoting greater patience, kindness, acceptance, and hospitality towards oneself and others. Additionally, mindfulness encourages greater fulfillment in daily life by focusing more on the present moment and reducing rumination, negative thoughts, and anxiety about the future.

In [9]:
# The_Mindfulness_Toolbox_for_Relationships
query_engine = mindfulness_TB_relationships_index.as_query_engine()
response = query_engine.query("How does mindfulness relate to relationships?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Mindfulness relates to relationships by engaging a sense of curiosity and openness, which helps individuals escape the ego-centered perspective and view others in a new way. It allows for a profound level of relating to both inner and outer environments. By practicing mindfulness, individuals can bring awareness, compassion, and openness into their relationships, leading to a sense of liberation and freedom from old habits and toxic ways of reacting. Mindfulness also provides practical tools for addressing issues that impact daily living and the stresses of the real world. In the 21st century, with the challenges posed by technology and social media, mindfulness can help counteract the negative effects on relationships and promote empathy and connection.

## Define Agent Tools

### Analyze image tool

In [10]:
from llama_index.tools import BaseTool, FunctionTool
from llama_index.multi_modal_llms.gemini import GeminiMultiModal
from llama_index.multi_modal_llms.generic_utils import load_image_urls
from typing import List

In [11]:
def analyze_image(img_urls: List[str]) -> str:
  """Calls our Gemini vision API to analyze the image and return a description of the text contained in the image.
      Returns: A string with a description of the image and the mood it conveys if any.

  Args:
      img_urls (List[str]): The URL of one or more images that convey the users mood
  """
  image_documents = load_image_urls(img_urls)

  gemini = GeminiMultiModal(model="models/gemini-pro-vision", api_key=userdata.get('GOOGLE_API_KEY'))

  complete_response = gemini.complete(
      prompt="Identify what you see in the image and what mood it conveys if any",
      image_documents=image_documents,
  )

  return complete_response

vision_tool = FunctionTool.from_defaults(fn=analyze_image)

### Mindfulness routine recomendation tool

In [12]:
from llama_index.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=challenging_child_engine,
        metadata=ToolMetadata(
            name="challenging_child",
            description=(
                "The Challenging Child Toolbox 75 Mindfulness Based Practices Tools and Tips for Therapists"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=mindfulness_TB_50_engine,
        metadata=ToolMetadata(
            name="mindfulness_TB_50",
            description=(
                "The Mindfulness Toolbox 50 Practical Tips Tools Handouts for Anxiety Depression Stress and Pain"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=mindfulness_TB_relationships_engine,
        metadata=ToolMetadata(
            name="mindfulness_TB_relationships",
            description=(
                "The Mindfulness Toolbox for Relationships 50 Practical Tips Tools Handouts for Building Compassionate Connections"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [13]:
tools = query_engine_tools + [vision_tool]

## Create LlamaIndex Agent

### System prompt to give our agent some personality

In [14]:
SYSTEM_PROMPT = """You are an emotional support assistant with the expertise of an experienced counselor. Your primary role is to assist the user by encouraging them to provide a drawing that conveys their mood.
You offer professional, friendly, and helpful guidance based on current counseling and mindfulness practices. Once you receive the image, interpret it to discern what the user might be feeling and confirm with them if your observation is correct.
If your interpretation does not align with their feelings, engage in a dialogue until you accurately understand their mood. Your knowledge is exclusively focused on understanding the user's emotions and recommending mindfulness routines using the tool, tailored to their mood.
Thus, you will only provide responses related to these areas. If a question falls outside your area of expertise or if you lack the necessary information, you will inform the user by saying,
'Sorry, I do not know the answer to your question.' and then prompt for more information related to their feelings. Once they confirm that you have correctly understood their feelings, your task is to recommend a suitable mindfulness routine using the tool."""

### Init our agent

In [30]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-4-1106-preview") # Using GPT-4 Turbo (Beta)

agent = OpenAIAgent.from_tools(
    tools,
    llm=llm,
    verbose=True,
    system_prompt=SYSTEM_PROMPT,
)

## Test Our Agent

I'm picturing the frontend give the user the option to chat or attach one or more images that would get uploaded to cloud storage.  Then our back could prompt the agent with something like "Here's a drawing of how I'm feeling {Image URL Here}."

For the demo below you need to manually paste the image URL into the chat (i.e. https://ih1.redbubble.net/image.3636044620.1142/bg,f8f8f8-flat,750x,075,f-pad,750x1000,f8f8f8.u5.jpg)

Single prompt example

In [31]:
response = agent.chat('hi')
display(Markdown(f"**Response:**\n\n{response}"))

STARTING TURN 1
---------------

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Hello! How can I assist you today? If you're feeling up to it, you can share a drawing that conveys your mood, and I'll do my best to understand and support you.

Multi-turn chat example

In [ ]:
response = agent.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

Human: hi
STARTING TURN 1
---------------

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Assistant: Hello! How can I assist you today? If you're feeling up to it, you can share a drawing that conveys your mood, and I'll do my best to understand and support you.

Human: https://ih1.redbubble.net/image.3636044620.1142/bg,f8f8f8-flat,750x,075,f-pad,750x1000,f8f8f8.u5.jpg
STARTING TURN 1
---------------

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Calling Function ===
Calling function: analyze_image with args: {"img_urls":["https://ih1.redbubble.net/image.3636044620.1142/bg,f8f8f8-flat,750x,075,f-pad,750x1000,f8f8f8.u5.jpg"]}
INFO:tornado.access:200 GET /v1beta/models/gemini-pro-vision?

## TODO:

### Reset Agent State

In [ ]:
# agent.reset()

In [ ]:
# response = agent.chat_repl()

### 2nd Session (Returning User)

Specify chat history with a summary of our last session or sessions.  This would need to be queried from our session history.

In [ ]:
# response = agent.chat_repl()